<a href="https://colab.research.google.com/github/absabry/Pyspark-tutorial/blob/master/2.%20schemas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schemas

## Download and install Spark

In [1]:
!ls # we're good, dont need to install

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [0]:
#!apt-get update
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#!pip install -q findspark

## Setup environment

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [0]:
import pandas as pd 

## Preprocessing Chicago's Reported Crime Data using Pandas and Spark

In [3]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

+-------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|5324759|   HM564643|2006-08-27 01:04:34|  029XX N PULASKI RD|1310|    CRIMINAL DAMAGE|         TO PROPERTY|           RESIDENCE| fal

In [6]:
data_pd = pd.read_csv('reported-crimes.csv')
data_pd.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,5324759,HM564643,08/27/2006 01:04:34 AM,029XX N PULASKI RD,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,31.0,21.0,14,1149217.0,1919198.0,2006,02/28/2018 03:56:25 PM,41.934206,-87.727045,"(41.934206478, -87.727044563)"
1,5324762,HN185785,02/19/2007 06:00:00 PM,062XX W CORNELIA AVE,0920,MOTOR VEHICLE THEFT,ATT: AUTOMOBILE,STREET,False,False,...,36.0,17.0,07,1134118.0,1922619.0,2007,02/28/2018 03:56:25 PM,41.943874,-87.782453,"(41.943873678, -87.782453204)"
2,5324766,HN184408,02/19/2007 11:02:12 AM,091XX S UNIVERSITY AVE,0460,BATTERY,SIMPLE,STREET,True,False,...,8.0,47.0,08B,1185475.0,1844606.0,2007,02/28/2018 03:56:25 PM,41.728741,-87.596151,"(41.728740563, -87.596150779)"
3,5324767,HN185514,02/19/2007 11:23:16 PM,038XX W NORTH AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,STREET,True,False,...,30.0,23.0,16,1150789.0,1910366.0,2007,02/28/2018 03:56:25 PM,41.909940,-87.721499,"(41.909940034, -87.721499025)"
4,5324768,HN184854,02/19/2007 03:30:00 PM,090XX S MARSHFIELD AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,21.0,73.0,08B,1166926.0,1844740.0,2007,02/28/2018 03:56:25 PM,41.729524,-87.664096,"(41.72952412, -87.664096049)"


#### Trying different ways of getting data from spark dataframe

In [7]:
rc.take(5)

[Row(ID='5324759', Case Number='HM564643', Date=datetime.datetime(2006, 8, 27, 1, 4, 34), Block='029XX N PULASKI RD', IUCR='1310', Primary Type='CRIMINAL DAMAGE', Description='TO PROPERTY', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='2523', District='025', Ward='31', Community Area='21', FBI Code='14', X Coordinate='1149217', Y Coordinate='1919198', Year='2006', Updated On='02/28/2018 03:56:25 PM', Latitude='41.934206478', Longitude='-87.727044563', Location='(41.934206478, -87.727044563)'),
 Row(ID='5324762', Case Number='HN185785', Date=datetime.datetime(2007, 2, 19, 18, 0), Block='062XX W CORNELIA AVE', IUCR='0920', Primary Type='MOTOR VEHICLE THEFT', Description='ATT: AUTOMOBILE', Location Description='STREET', Arrest='false', Domestic='false', Beat='1633', District='016', Ward='36', Community Area='17', FBI Code='07', X Coordinate='1134118', Y Coordinate='1922619', Year='2007', Updated On='02/28/2018 03:56:25 PM', Latitude='41.943873678', Longitude='-

In [8]:
rc.head(5)

[Row(ID='5324759', Case Number='HM564643', Date=datetime.datetime(2006, 8, 27, 1, 4, 34), Block='029XX N PULASKI RD', IUCR='1310', Primary Type='CRIMINAL DAMAGE', Description='TO PROPERTY', Location Description='RESIDENCE', Arrest='false', Domestic='false', Beat='2523', District='025', Ward='31', Community Area='21', FBI Code='14', X Coordinate='1149217', Y Coordinate='1919198', Year='2006', Updated On='02/28/2018 03:56:25 PM', Latitude='41.934206478', Longitude='-87.727044563', Location='(41.934206478, -87.727044563)'),
 Row(ID='5324762', Case Number='HN185785', Date=datetime.datetime(2007, 2, 19, 18, 0), Block='062XX W CORNELIA AVE', IUCR='0920', Primary Type='MOTOR VEHICLE THEFT', Description='ATT: AUTOMOBILE', Location Description='STREET', Arrest='false', Domestic='false', Beat='1633', District='016', Ward='36', Community Area='17', FBI Code='07', X Coordinate='1134118', Y Coordinate='1922619', Year='2007', Updated On='02/28/2018 03:56:25 PM', Latitude='41.943873678', Longitude='-

In [9]:
rc.limit(5)

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

## Schemas

In [10]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [0]:
from pyspark.sql.types import StructField, StructType, StringType, TimestampType, BooleanType, DoubleType, IntegerType

In [0]:
boolean_labels = ['Domestic']
timestamp_labels = ['Date']
double_labels = ['Latitude', 'Longitude']
int_labels = ['Year']
string_labels = list(set(rc.columns) - set(boolean_labels) - set(timestamp_labels) - set(double_labels) - set(int_labels))

In [0]:
schema = []
for x in rc.columns : 
  if x in boolean_labels: 
    schema.append(StructField(x, BooleanType(), True))
  if x in string_labels: 
    schema.append(StructField(x, StringType(), True))
  if x in int_labels: 
    schema.append(StructField(x, IntegerType(), True))
  if x in double_labels: 
    schema.append(StructField(x, DoubleType(), True))
  if x in timestamp_labels: 
    schema.append(StructField(x, TimestampType(), True))
schema = StructType(schema)

In [40]:
rc = spark.read.csv('reported-crimes.csv',schema=schema)
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [41]:
rc.show(5) # we have null values, that means something is wrong with the schema or with data

+----+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|  ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+----+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|null|       null|null| null|null|        null|       null|                null|  null|    null|null|    null|null|          null|    null|        null|        null|null|      null|    null|     null|    null|
|null|       null|null| null|null|        null|       null|                null|  null|    null|null|    null|null|          null|    null|        null|        

In [42]:
rc.select('ID', 'District').show(5)

+-------+--------+
|     ID|District|
+-------+--------+
|     ID|District|
|5324759|     025|
|5324762|     016|
|5324766|     004|
|5324767|     025|
+-------+--------+
only showing top 5 rows



In [0]:
rc = rc.withColumnRenamed('Description', 'Desc'); # change column name

In [47]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Desc',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [0]:
rc_without_ID = rc.drop('ID')

In [49]:
rc_without_ID.columns

['Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Desc',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']